# EPL Raw Data Validation — Analysis

### Notebook Purpose


This notebook is created to analyse the validation output genearated during the raw EPL data validation step. 
The goal is to identify and understand issues raised during the validation process, particularly ERROR-level issues that block the  processed data build phase. 

Insights from this analysis are used to to design and finalize the processed dataset schema. 

No data transformation is performed here.



In [2]:
import pandas as pd
import json

from pathlib import Path

In [3]:
PARENT_ROOT = Path.cwd().parents[0]
validation_log_path = PARENT_ROOT / "logs" / "validation_report.json"


with open(validation_log_path, "r") as json_file:
    report = json.load(json_file)

issues = report["issues"]
metadata = report["run_metadata"]

In [4]:
df = pd.DataFrame(issues)

df.head()

,severity,season,file,issue_type,column,expected,actual,message
0,ERROR,2014_15,season_1415.csv,missing_required_column,AvgH,True,False,Required column is missing
1,ERROR,2014_15,season_1415.csv,missing_required_column,AvgD,True,False,Required column is missing
2,ERROR,2014_15,season_1415.csv,missing_required_column,AvgA,True,False,Required column is missing
3,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,HHW,True,False,Optional column for post-match diagnosis is mi...
4,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,AHW,True,False,Optional column for post-match diagnosis is mi...


### Validation Summary

In [5]:
df[["issue_type", "severity"]].value_counts()

issue_type                          severity
missing_optional_diagnostic_column  WARNING     88
missing_required_column             ERROR       15
Name: count, dtype: int64

In [6]:
metadata

{'run_timestamp': '2025-12-26 12:42:12',
 'total_files_checked': 11,
 'files_passed': 6,
 'files_failed': 5}

**Insightings - Validation**

- Total Season Checked: 11
- Files Padded Validation: 6
- Files Failed Validation: 5
- Primary failure categories: Missing required colum (ERROR) - 15

### ERROR-Level Issues (BLOCKING)

In [7]:
blocking_failures = df.loc[df["severity"] == "ERROR"]

blocking_failures

,severity,season,file,issue_type,column,expected,actual,message
0,ERROR,2014_15,season_1415.csv,missing_required_column,AvgH,True,False,Required column is missing
1,ERROR,2014_15,season_1415.csv,missing_required_column,AvgD,True,False,Required column is missing
2,ERROR,2014_15,season_1415.csv,missing_required_column,AvgA,True,False,Required column is missing
11,ERROR,2015_16,season_1516.csv,missing_required_column,AvgH,True,False,Required column is missing
12,ERROR,2015_16,season_1516.csv,missing_required_column,AvgD,True,False,Required column is missing
13,ERROR,2015_16,season_1516.csv,missing_required_column,AvgA,True,False,Required column is missing
22,ERROR,2016_17,season_1617.csv,missing_required_column,AvgH,True,False,Required column is missing
23,ERROR,2016_17,season_1617.csv,missing_required_column,AvgD,True,False,Required column is missing
24,ERROR,2016_17,season_1617.csv,missing_required_column,AvgA,True,False,Required column is missing
33,ERROR,2017_18,season_1718.csv,missing_required_column,AvgH,True,False,Required column is missing


In [8]:
blocking_failures.groupby("season").size()

season
2014_15    3
2015_16    3
2016_17    3
2017_18    3
2018_19    3
dtype: int64

In [9]:
blocking_failures.groupby("column").size()

column
AvgA    5
AvgD    5
AvgH    5
dtype: int64

**Insights - ERROR-Level Issues**

- **Missing Core Pre-Match Features**:
  - 3 required columns are missing: AvgH, AvgD, AvgA
  - These columns represent pre-match market-average odds, which are core predictive signals.
  - These columns are missing for seasons 2014_15 to 2018_19 duue to limitations in the source dataset.
- **Decison**: 
  - Seasons 2014_15 to 2018_19 are excluded.
  - Pre-match market-average odds are not consistently available, they're are required for both feature consistency and interpretability.
  - Reconstructing or approximating these odds would alter feature semantics and introduce unverifiable assumptions.
  - Therefore the dataset is restricted to seasons with complete, comparable pre-match market information.

### WARNING-Level Issues (Non-blocking)

In [10]:
non_blocking_falures = df[df["severity"] != "ERROR"]
non_blocking_falures

,severity,season,file,issue_type,column,expected,actual,message
3,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,HHW,True,False,Optional column for post-match diagnosis is mi...
4,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,AHW,True,False,Optional column for post-match diagnosis is mi...
5,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,HFKC,True,False,Optional column for post-match diagnosis is mi...
6,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,AFKC,True,False,Optional column for post-match diagnosis is mi...
7,WARNING,2014_15,season_1415.csv,missing_optional_diagnostic_column,HO,True,False,Optional column for post-match diagnosis is mi...
...,...,...,...,...,...,...,...,...
98,WARNING,2024_25,season_2425.csv,missing_optional_diagnostic_column,AFKC,True,False,Optional column for post-match diagnosis is mi...
99,WARNING,2024_25,season_2425.csv,missing_optional_diagnostic_column,HO,True,False,Optional column for post-match diagnosis is mi...
100,WARNING,2024_25,season_2425.csv,missing_optional_diagnostic_column,AO,True,False,Optional column for post-match diagnosis is mi...
101,WARNING,2024_25,season_2425.csv,missing_optional_diagnostic_column,HBP,True,False,Optional column for post-match diagnosis is mi...


In [16]:
non_blocking_falures["season"].value_counts()

season
2014_15    8
2015_16    8
2016_17    8
2017_18    8
2018_19    8
2019_20    8
2020_21    8
2021_22    8
2022_23    8
2023_24    8
2024_25    8
Name: count, dtype: int64

In [12]:
non_blocking_falures.groupby("column").size()

column
ABP     11
AFKC    11
AHW     11
AO      11
HBP     11
HFKC    11
HHW     11
HO      11
dtype: int64

In [21]:
# columns missing in each season
missing_col_df = non_blocking_falures[["season", "column"]]

In [23]:
missing_col_df.loc[missing_col_df["season"] == "2014_15"]

,season,column
3,2014_15,HHW
4,2014_15,AHW
5,2014_15,HFKC
6,2014_15,AFKC
7,2014_15,HO
8,2014_15,AO
9,2014_15,HBP
10,2014_15,ABP


In [24]:
missing_col_df.loc[missing_col_df["season"] == "2024_25"]

,season,column
95,2024_25,HHW
96,2024_25,AHW
97,2024_25,HFKC
98,2024_25,AFKC
99,2024_25,HO
100,2024_25,AO
101,2024_25,HBP
102,2024_25,ABP


**Insights – WARNING-Level Issues**

- **Missing Optional Post-Match Diagnostic Features:**

  - The following columns are consistently missing across all validated seasons:

      - HHW, AHW (Hit Woodwork)

      - HFKC, AFKC (Free Kicks Conceded)

      - HO, AO (Offsides)

      - HBP, ABP (Booking Points)

  - These columns are post-match statistics intended for diagnostic and exploratory analysis.

  - Validation results indicate that these features are not provided by the source dataset for the selected seasons.

- **Decision:**

  - These columns are excluded from the processed dataset schema.

  - Their absence does not impact model training, as they are not used as predictive inputs.

  - Retaining structurally absent columns would introduce unnecessary sparsity and reduce schema clarity.

  - The processed dataset therefore includes only optional diagnostic features that are consistently available.

### Insights – INFO-Level Issues

- No INFO-level issues were detected during validation.

- Row counts across all validated seasons fall within the expected range for Premier League matches.

- No additional data sanity concerns were identified at this stage.

### Schema Decisions


- **Scope of Included Seasons**

    Missing pre-match market-average odds (AvgH, AvgD, AvgA). Only seasons with complete core pre-match features are included. *Seasons 2014_15 to 2018_19* are excluded because of the absence of required features.

- **Derived `Season` Column**

    A new column named `season` will be added to the processed dataset to represent the competition season (e.g. 2019_20).
    This column is not present in the raw source files and will be added during processing to support temporal analysis, season-wise evaluation, and reproducibility.The season value is derived from the file itself and does not affect the original data.

- **Core Feature Set**

    The following features are divided into three categories - **Identifiers, Pre-match features, Targets**.
    These are the features from the dataset that are required for model training and they're consistently available across all included seasons.

  - **Identifiers:**
    - Date

    - Season

    - HomeTeam

    - AwayTeam

  - **Pre-match features:**

    - AvgH

    - AvgD

    - AvgA

  - **Targets:**

    - FTHG

    - FTAG

    - FTR

- **Optional Diagnostic Features (Included)**

    Post-match statistics that are consistently available across included seasons are included for diagnostic and evaluation purposes. These features are not used for prediction and they will be used for model evaluatin, and error analysis.

    Examples include:

  - Shots

  - Corners

  - Fouls

  - Cards

  - Half-time goals

- **Optional Diagnostic Features (Excluded)**

  - The following features are excluded due to absence across all validated seasons:

    - HHW, AHW

    - HFKC, AFKC

    - HO, AO

    - HBP, ABP

    They're excluded to avoid structurally empty columns

- **Excluded Data Categories**

  The following data categories are excluded from the processed dataset:

  - Individual bookmaker odds

  - Asian handicap odds

  - Over/Under odds

    These signals are redundant given market-average odds and they significantly increase dimensionality without any interpretability benefit.

- **Data Integrity Guarantees**

    For all included seasons:

  - Each season contains approximately 380 matches.

  - Team naming is consistent.

  - Headers are normalized.

    These guarantees are enforced via the raw data validation scripts.


This schema will be implemented in the processed dataset build step.